In [2]:
import pandas as pd

# Read data from files 
train = pd.read_csv( "labeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )
test = pd.read_csv( "testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print("Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size ))

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [7]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review,"lxml").get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [8]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
nltk.download()   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [9]:
sentences = []  # Initialize an empty list of sentences

print ("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print ("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


C:\Users\Hawkeye\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
C:\Users\Hawkeye\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'...'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
C:\Users\Hawkeye\Anaconda3\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\Hawkeye\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'music.'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle int

Parsing sentences from unlabeled set


C:\Users\Hawkeye\Anaconda3\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\Hawkeye\Anaconda3\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\Hawkeye\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'... ...'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
C:\Users\Hawke

In [12]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

C:\Users\Hawkeye\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2017-08-06 10:24:20,141 : INFO : 'pattern' package not found; tag filters are not available for English
2017-08-06 10:24:20,155 : INFO : collecting all words and their counts
2017-08-06 10:24:20,157 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-08-06 10:24:20,232 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2017-08-06 10:24:20,310 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types


Training model...


2017-08-06 10:24:20,402 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types
2017-08-06 10:24:20,486 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2017-08-06 10:24:20,546 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2017-08-06 10:24:20,632 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2017-08-06 10:24:20,695 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2017-08-06 10:24:20,780 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2017-08-06 10:24:20,864 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2017-08-06 10:24:20,961 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2017-08-06 10:24:21,083 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 

In [13]:
model.doesnt_match("man woman child kitchen".split())

'kitchen'

In [14]:
model.doesnt_match("france england germany berlin".split())

'berlin'

In [16]:
model.doesnt_match("paris berlin london austria".split())

'berlin'

In [17]:
model.most_similar("man")

[('woman', 0.6176169514656067),
 ('lady', 0.5938267707824707),
 ('lad', 0.5518506765365601),
 ('monk', 0.5332566499710083),
 ('guy', 0.5223327875137329),
 ('farmer', 0.5106891393661499),
 ('doctor', 0.5073086023330688),
 ('person', 0.5018740296363831),
 ('men', 0.5014002919197083),
 ('businessman', 0.4967166781425476)]

In [18]:
model.most_similar("awful")

[('terrible', 0.7703068852424622),
 ('atrocious', 0.724236786365509),
 ('dreadful', 0.7165095806121826),
 ('horrible', 0.7153354287147522),
 ('abysmal', 0.7045223712921143),
 ('horrendous', 0.6898051500320435),
 ('appalling', 0.6853083372116089),
 ('horrid', 0.6733729839324951),
 ('lousy', 0.6323549747467041),
 ('amateurish', 0.6066460609436035)]

In [29]:
model.most_similar("japanese")

[('chinese', 0.6904727220535278),
 ('foreign', 0.6114503741264343),
 ('spanish', 0.6082684993743896),
 ('korean', 0.6058086156845093),
 ('european', 0.6051000356674194),
 ('asian', 0.6050255298614502),
 ('indian', 0.6006872653961182),
 ('norwegian', 0.5854333639144897),
 ('german', 0.5753902792930603),
 ('native', 0.572583794593811)]